# HW1 Triage

In [1]:
// TypeScript Jupyter extension
import * as tslab from "tslab";

// CSC 600 Libraries
import { drawList, drawTree, drawCallStack, drawStackTrace, requireCytoscape, requireCarbon} from "./lib/draw";
import * as list from "./lib/list";
import * as tree from "./lib/tree";

requireCarbon();
requireCytoscape();

## Problem 3: Hybrid List and Tree

We saw lists and binary trees in class. If the problem we are working on is a list
or tree, then we can reuse generic list and tree data-types. However, the problem
at hand may not be exactly a list or tree, in which case we will need to define our
own data-types. In this problem, we will practice defining our own data-types. We
will look at hyrbid lists and trees where a tree can have one or two children. We
give the BNF below.

$$
<hybrid> ::= 
    \texttt{HLeaf}
  \,\,|\,\, \texttt{OneChild}(x, <hybrid>)
  \,\,|\,\, \texttt{TwoChild}(x, <hybrid>, <hybrid>)
$$

In [2]:
// List definition as a reminder
enum _List { NIL, CONS };
type List<T> = {tag: _List.NIL} | {tag: _List.CONS, contents: T, rest: List<T>};

function Nil<T>(): List<T> {
    return {tag: _List.NIL};
}

function Cons<T>(x: T, ls: List<T>): List<T> {
    return {tag: _List.CONS, contents: x, rest: ls};
}

In [3]:
// Tree definition as a reminder
enum _Tree { LEAF, NODE };
type Tree<T> = 
{tag: _Tree.LEAF} | 
{tag: _Tree.NODE, contents: T, left: Tree<T>, right: Tree<T>};

function Leaf<T>(): Tree<T> {
    return {tag: _Tree.LEAF};
}

function Node<T>(x: T, left: Tree<T>, right: Tree<T>): Tree<T> {
    return {tag: _Tree.NODE, contents: x, left: left, right: right};
}

 function LeafNode<T>(x: T): Tree<T> {
    return Node(x, Leaf(), Leaf());
}

In [4]:
// List definition as a reminder
enum _List { NIL, CONS };
type List<T> = {tag: _List.NIL} | {tag: _List.CONS, contents: T, rest: List<T>};

function Nil<T>(): List<T> {
    return {tag: _List.NIL};
}

function Cons<T>(x: T, ls: List<T>): List<T> {
    return {tag: _List.CONS, contents: x, rest: ls};
}
// Tree definition as a reminder
enum _Tree { LEAF, NODE };
type Tree<T> = 
{tag: _Tree.LEAF} | 
{tag: _Tree.NODE, contents: T, left: Tree<T>, right: Tree<T>};

function Leaf<T>(): Tree<T> {
    return {tag: _Tree.LEAF};
}

function Node<T>(x: T, left: Tree<T>, right: Tree<T>): Tree<T> {
    return {tag: _Tree.NODE, contents: x, left: left, right: right};
}

 function LeafNode<T>(x: T): Tree<T> {
    return Node(x, Leaf(), Leaf());
}

### Problem 3a:

Complete the data-type definition below.

In [5]:
enum _Hybrid { HLEAF, ONECHILD, TWOCHILD };
type Hybrid<T> =
    {tag: _Hybrid.HLEAF}  // TODO: implement the rest
  // | OneChild(x, <hybrid>)
  | {tag: _Hybrid.ONECHILD, contents: T, child: Hybrid<T>}
  // | TwoChild(x, <hybrid>, <hybrid>)
  | {tag: _Hybrid.TWOCHILD, contents: T, left: Hybrid<T>, right: Hybrid<T>};


function HLeaf<T>(): Hybrid<T> {
    // NOTE: this is similar to List NIL or Tree LEAF
    return { tag: _Hybrid.HLEAF }; // This one is completed
}

function OneChild<T>(x: T, child: Hybrid<T>): Hybrid<T> {
    // NOTE: this is similar to List CONS
    return { tag: _Hybrid.ONECHILD, contents: x, child: child };
}

function TwoChild<T>(x: T, left: Hybrid<T>, right: Hybrid<T>): Hybrid<T> {
    // NOTE: this is similar to Tree NODE
    return { tag: _Hybrid.TWOCHILD, contents: x, left: left, right: right };
}

### Problem 3b:

Implement the following two structures in the `Hybrid` ADT.

hybrid1
```
    3
   / \
  2   5
  |   |
  1   4
```

hybrid2
```
     4
    / \
   /   \
  2     6
 / \    |
1   3   5
```

In [6]:
// Solution: build each hybrid data structure from the bottom up
const h1 = OneChild(1, HLeaf())
const h4 = OneChild(4, HLeaf())
const h2 = OneChild(2, h1)
const h5 = OneChild(5, h4)
const h3 = TwoChild(3, h2, h5)
const hybrid1 = h3

const hybrid2 = TwoChild(4, TwoChild(2, OneChild(1, HLeaf()), OneChild(3, HLeaf())), OneChild(6, HLeaf()));

### Problem 3c

You may have noticed that the Hybrid ADT was able to encode
the "same" structures as Tree data structure. That the "same"
data can be stored in a variety of equivalent encodings can
occur in practice.

Write a **recursive** function that converts a Hybrid data
structure into a Tree structure. When translating a node with
one child into a node, put the child in the left child.

In [7]:
function hybridToTree<T>(hy: Hybrid<T>): Tree<T> {
    switch (hy.tag) {
        case _Hybrid.HLEAF: {
            return Leaf();
        }
        case _Hybrid.ONECHILD: {
            // We put ONECHILD's child in the left child.
            return Node(hy.contents, hybridToTree(hy.child), Leaf());
        }
        case _Hybrid.TWOCHILD: {
            // This case is essentially an identity function because a TWOCHILD is a Tree NODE/
            return Node(hy.contents, hybridToTree(hy.left), hybridToTree(hy.right));
        }
    }
}

In [8]:
// Test to see that we got it right
drawTree(hybridToTree(hybrid1))

## Problem 4: JSON

You may be familiar with a concept known as a **webscraper** that:

1. Visits a web-page
2. *Recursively* visits each link on that page

This might produce a JSON data-structure such as

```
[
  {
    "url": "www.foobar.com",
    "links": [ { links json ... }]
  },
  ...,
  {
    "url": "www.bazbae.com",
    "links": [ { links json ... }]
  },
]
```

In [9]:
type JSONValue = null | string | JSONValue[] | JSONObject;
type JSONObject = { [key: string]: JSONValue };  // see JSON Spec, Object ::= { members }


const jsonLinkExample: JSONValue = [
    {
        "url": "one.com",
        "links": [ 
            {
                "url": "two.com", 
                "links": [],
            },
            {
                "url": "three.com", 
                "links": [],
            }
        ]
    },
    {
        "url": "four.com",
        "links": [ 
            {
                "url": "seven.com", 
                "links": [
                    {
                        "url": "one.com",
                        "links": [
                            {
                                "url": "eight.com",
                                "links": []
                            }
                        ]
                    }
                ],
            },
            {
                "url": "three.com", 
                "links": [],
            }
        ]
    }
]

In [10]:
type JSONValue =         // A JSONValue is either a
    null                       // 1) null
    | string                      // 2) string
    | JSONValue[]                 // 3) array of JSONValues
    | JSONObject                  // 4) JSONObject
    ;

type JSONObject = {
    [key: string]: JSONValue    // Dictionary with string keys and JSONValue values
};

const jsonLinkExample2: JSONValue = [
    {
        "authority": "one.com",
        "path": "1",
        "links": [
            {
                "authority": "www.two.com",
                "links": [],
            },
            {
                "authority": "three.com",
                "path": "",
                "links": [],
            }
        ]
    },
    {
        "authority": "www.four.com",
        "links": [
            {
                "authority": "seven.com",
                "links": [
                    {
                        "authority": "app.one.com",
                        "path": "2",
                        "links": [
                            {
                                "authority": "eight.com",
                                "links": []
                            }
                        ]
                    }
                ],
            },
            {
                "authority": "app.three.com",
                "path": "locations/42",
                "links": [],
            }
        ]
    }
]

### Problem 4a

Write a **recursive** function that creates an array of all
URLs (duplicates included) found in the JSON.

In [11]:
type JSONValue = null | string | JSONValue[] | { [key: string]: JSONValue };

function allURL(json: JSONValue): string[] {
    if (json === null) {
        // know json is null
        return [];
    } else if (typeof json === "string") {
        // know json is string
        return [];
    } else if (Array.isArray(json)) {
        // know json is array with type JSONValue[]
        let acc = [];
        for (const jsonVal of json) {
            acc = acc.concat(allURL(jsonVal));
        }
        return acc;
    }
    else {
        // know json is { [key: string]: JSONValue }
        const jsonObj = json as { url: string, links: JSONValue };
        return [jsonObj.url].concat(allURL(jsonObj.links));
    }
}

allURL(jsonLinkExample)

[
  'one.com',
  'two.com',
  'three.com',
  'four.com',
  'seven.com',
  'one.com',
  'eight.com',
  'three.com'
]


In [12]:
type JSONValue = null | string | JSONValue[] | { [key: string]: JSONValue };

function allURL(json: JSONValue): string[] {
    if (json === null) {
        // know json is null
        return [];
    } else if (typeof json === "string") {
        // know json is string
        return [];
    
    } else if (Array.isArray(json)) {
        // know json is array with type JSONValue[]
        let acc = [];
        for (const jsonVal of json) {
            acc = acc.concat(allURL(jsonVal));
        }
        return acc;
    }
    else {
        // know json is { [key: string]: JSONValue }
        const jsonObj = json as {authority: string, links: JSONValue };
        return [jsonObj.authority].concat(allURL(jsonObj.links));
    }
}

allURL(jsonLinkExample2)

[
  'one.com',
  'www.two.com',
  'three.com',
  'www.four.com',
  'seven.com',
  'app.one.com',
  'eight.com',
  'app.three.com'
]


In [13]:
type JSONValue = null | string | JSONValue[] | { [key: string]: JSONValue };

function allURL(json: JSONValue): string[] {
    if (json === null) {
        // know json is null
        return [];
    } else if (typeof json === "string") {
        // know json is string
        return [];
    } else if (Array.isArray(json)) {
        // know json is array with type JSONValue[]
        let acc = [];
        for (const jsonVal of json) {
            acc = acc.concat(allURL(jsonVal));
        }
        return acc;
    }
    else {
        // know json is { [key: string]: JSONValue }
        const jsonObj = json as { url: string, links: JSONValue };
        return [jsonObj.url].concat(allURL(jsonObj.links));
    }
}

allURL(jsonLinkExample)

[
  'one.com',
  'two.com',
  'three.com',
  'four.com',
  'seven.com',
  'one.com',
  'eight.com',
  'three.com'
]


In [ ]:
const y1 = 3;
function f(x1: number): number {
    function g(x2: number, y2: number) {
         return x2 + y2;
    }
    return g(x1, 1);
}
f(1)

In [ ]:
const mathArray: ("plus" | "minus" | "times" | "divide")[] = [];
    function plus(x: number, y: number) {
        return x + y
    }
    function minus(x: number, y: number) {
        return x - y
    }
    function times(x: number, y: number) {
        return x * y
    }
    function divide(x: number, y: number) {
        return x / y
    }
    for (var i = 0; i < names.length; i++) {
        //mathArray.push(names[i])
        switch (names[i]) {
            case "plus":
                //console.log("plus")
                numbers.push("+")
                //(x:number, y:number)=>x+y
                break;
            case "minus":
                //console.log("minus")
                numbers.push("-")
                //(x:number, y:number)=>x-y
                break;
            case "times":
                //console.log("times")
                numbers.push("*")
                //(x:number, y:number)=>x*y
                break;
            case "divide":
                //console.log("divide")
                numbers.push("/")
                //(x:number, y:number)=>x/y
                break;

        }
        
    }
function funct(list: [],x:number,y:number):number{
        for (var i = 0; i < names.length; i++) {
            if (list[i] == "plus"){
                return x+y;
            }
        }
    }
    // print out mathArray

### Problem 4b

Given a JSON as above where "links" contains the same JSON
format, arbitrarily nested, write a **recursive** function
that counts the number of times an exact match of a given url
occurs in the JSON object.

In [ ]:
type JSONValue = null | string | JSONValue[] | { [key: string]: JSONValue };

function recCountURL(url: string, json: JSONValue): number {
    if (json === null) {
        // know json is null
        return 0; // [];
    } else if (typeof json === "string") {
        // know json is string
        return 0; // [];
    } else if (Array.isArray(json)) {
        // know json is array with type JSONValue[]
        let acc = 0;
        for (const jsonVal of json) {
            acc += recCountURL(url, jsonVal);
        }
        return acc;
    }
    else {
        // know json is { [key: string]: JSONValue }
        const jsonObj = json as { url: string, links: JSONValue };
        if (jsonObj.url === url) {
            return 1 + recCountURL(url, jsonObj.links);
        } else {
            return recCountURL(url, jsonObj.links);
        }
    }
}

console.log(recCountURL("one.com", jsonLinkExample))
console.log(recCountURL("two.com", jsonLinkExample))
console.log(recCountURL("four.com", jsonLinkExample))

In [ ]:
function recCountURL(authority: string, json: JSONValue): number {
    let count = 0;

    if (Array.isArray(json)) {
        for (const entry of json) {
            count += recCountURL(authority, entry);
        }
    } else if (json !== null && typeof json !== "string") {
        if (json.authority === authority) {
            count += 1;
        }

        const sublinks = json.links;
        if (sublinks !== undefined) {
            count += recCountURL(authority, sublinks);
        }
    }

    return count;
}
console.log(recCountURL("one.com", jsonLinkExample2))
console.log(recCountURL("two.com", jsonLinkExample2))
console.log(recCountURL("four.com", jsonLinkExample2))

### Problem 4c

Do the same as problem 4b, but this time with an **iterative**
function.

In [ ]:
function iterCountURL(url: string, json: JSONValue): number {
    let callStack: JSONValue[] = [json];
    let acc = 0;
    while (callStack.length > 0) {
        let json = callStack.pop();
        if (json === null) {
            acc += 0;
        } else if (typeof json === "string") {
            acc += 0;
        } else if (Array.isArray(json)) {
            for (const jsonVal of json) {
                callStack.push(jsonVal);
            }
        } 
        else { // Object case
            const jsonObj = json as { url: string, links: JSONValue };
            callStack.push(jsonObj.links);
            if (jsonObj.url === url) {
                acc += 1
            }
        }
    }
    return acc;
}

console.log(iterCountURL("one.com", jsonLinkExample))
console.log(iterCountURL("two.com", jsonLinkExample))
console.log(iterCountURL("three.com", jsonLinkExample))

## Problem 2

We'll start with 2b.

### Problem 2b:

Previously we converted a tree into a list. Now we will convert a list into an array.

Example 1:

List
```
1 -> 2 -> 3 -> 4
```

Array
```
[1, 2, 3, 4]
```

In [ ]:
function listToArr<T>(ls: List<T>): T[] {
    switch (ls.tag) {
        case _List.NIL: {
            return [];
        }
        case _List.CONS: {
            return [ls.contents].concat(listToArr(ls.rest));
        }
    }
}

console.log(listToArr(Cons(1, Cons(2, Cons(3, Cons(4, Nil()))))))

### Problem 2c:

In this problem we'll convert an array into a tree.

- Example 1:

Array
```
[1, 2, 3]
```

Tree
```
    2
   / \
  1   3
```

- Example 2:

Array
```
[1, 2, 3, 4]
```

Tree
```
    3
   / \
  2   4
 /    
1
```

- Example 3:

Array
```
[1, 2, 3, 4, 5]
```

Tree
```
    3
   / \
  2   5
 /   /
1    4
```

- Example 4:

Array
```
[1, 2, 3, 4, 5, 6]
```

Tree
```
     4
    / \
   /   \
  2     6
 / \    /
1   3  5
```

In [ ]:
function arrayToTree<T>(arr: T[]): Tree<T> {
    if (arr.length == 0) {
        return Leaf();
    } else if (arr.length == 1) {
        return LeafNode(arr[0]);
    } else {
        const midpt = Math.trunc(arr.length / 2);
        return Node(arr[midpt], arrayToTree(arr.slice(0, midpt)), arrayToTree(arr.slice(midpt + 1)))
    }
}

In [ ]:
drawTree(arrayToTree([1, 2, 3,4,5]))

In [ ]:
drawTree(arrayToTree([1, 2, 3, 4]))

In [ ]:
drawTree(arrayToTree([1, 2, 3, 4, 5]))

In [ ]:
drawTree(arrayToTree([1, 2, 3, 4, 5, 6]))

### Problem 2a: (Apparently very difficult)

Convert a tree into a list using "mirrored" postfix ordering.
That is, we'll visit the right child, the left child, and then
finally the current node. (In standard postfix, we'll visit the
left, the right, and then the current node.)


- Example 1: 

Original 
```
   1
  / \
 2   3
```

Postfix
```
2 -> 3 -> 1
```

Mirrored Postfix
```
3 -> 2 -> 1
```

- Example 2:

Original 
```
   1
  / \
 2   3
 |  / \
 4  5  6
```

Postfix
```
4 -> 2 -> 5 -> 6 -> 3 -> 1
```

Mirrored Postfix
```
6 -> 5 -> 3 -> 4 -> 2 -> 1
```

In [ ]:
const example1 = Node(1, LeafNode(2), LeafNode(3))
const example2 = Node(1, Node(2, LeafNode(4), Leaf()), Node(3, LeafNode(5), LeafNode(6)))

In [ ]:
/*
// List definition as a reminder
enum _List { NIL, CONS };
type List<T> = {tag: _List.NIL} | {tag: _List.CONS, contents: T, rest: List<T>};

function Nil<T>(): List<T> {
    return {tag: _List.NIL};
}

function Cons<T>(x: T, ls: List<T>): List<T> {
    return {tag: _List.CONS, contents: x, rest: ls};
}
// Tree definition as a reminder
enum _Tree { LEAF, NODE };
type Tree<T> = 
{tag: _Tree.LEAF} | 
{tag: _Tree.NODE, contents: T, left: Tree<T>, right: Tree<T>};

function Leaf<T>(): Tree<T> {
    return {tag: _Tree.LEAF};
}

function Node<T>(x: T, left: Tree<T>, right: Tree<T>): Tree<T> {
    return {tag: _Tree.NODE, contents: x, left: left, right: right};
}

 function LeafNode<T>(x: T): Tree<T> {
    return Node(x, Leaf(), Leaf());
}
*/

In [ ]:
function append<T>(ls1: List<T>, ls2: List<T>): List<T> {
    switch (ls1.tag) {
        case _List.NIL: {
            return ls2;
        }
        case _List.CONS: {
            return Cons(ls1.contents, append(ls1.rest, ls2));
        }
    }
}

function mirroredPostfix<T>(t: Tree<T>): List<T> {
    switch (t.tag) {
        case _Tree.LEAF: {
            return Nil();
        }
        case _Tree.NODE: {
            const left = mirroredPostfix(t.left);
            const right = mirroredPostfix(t.right);
            return append(append(right, left), Cons(t.contents, Nil()));
        }
    }
}

In [ ]:
drawList(mirroredPostfix(example1))

In [ ]:
drawList(mirroredPostfix(example2))

In [ ]:
function mirroredPostfix2<T>(t: Tree<T>): List<T> {
    function go(t: Tree<T>, rest: List<T>): List<T> {
        switch (t.tag) {
            case _Tree.LEAF: {
                return rest;
            }
            case _Tree.NODE: {
                // Note: preorder in reverse
                return go(t.right, go(t.left, Cons(t.contents, rest)));
            }
        }
    }
    
    return go(t, Nil())
}

In [ ]:
drawList(mirroredPostfix2(example1))

In [ ]:
drawList(mirroredPostfix2(example2))

### Problem 2d:

In summary, we saw we could convert trees into lists, lists into
arrays, and arrays back into trees. Consequently, an algorithm on
trees can be applied to arrays. For example, in class, we saw
that we could compute the smallest element in an array iteratively
and recursively.

Write a **recursive** function that finds the smallest element in a
tree of numbers. If it is an empty Tree, return NaN. We can then
find the smallest number in a tree using this  implementation as in
```
function smallestElementArr2(arr: number[]): number {
    return smallestTree(arrayToTree(arr));
}
```

In [ ]:
function smallestElementTree(tr: Tree<number>): number {
    switch (tr.tag) {
        case _Tree.LEAF: {
            return NaN;
        }
        case _Tree.NODE: {
            let smallest = tr.contents;
            if (tr.left.tag != _Tree.LEAF) {
                smallest = Math.min(smallest, smallestElementTree(tr.left));
            }
            if (tr.right.tag != _Tree.LEAF) {
                smallest = Math.min(smallest, smallestElementTree(tr.right));
            }
            return smallest;
        }
    }
}

In [ ]:
smallestElementTree(Leaf())

In [ ]:
smallestElementTree(example1)

In [ ]:
smallestElementTree(example2)

In [ ]:
class Counter {
 private count: number;

 constructor() {
 this.count = 0;
 }

 increment() {
 this.count += 1;
 return this.count;
 }

 getCount() {
 return this.count;
 }
}


In [ ]:
type FunCounter = {increment: () => void, getCount: () => number};
function makeCounter(): FunCounter {
 let count = 0;

 function _increment() {
 // Increment **captures** the variable count
 count += 1;
 return count;
 }

 function _getCount() {
 return count;
 }

 return {increment: _increment, getCount: _getCount};
}


In [ ]:

const counter = makeCounter();
console.log(counter.increment());
console.log(counter.increment());
console.log(counter.increment());


In [ ]:
class UnsafePair<S, T> {
    public fst: S;
    public snd: T;

    constructor(fst: S, snd: T) {
        this.fst = fst;
        this.snd = snd;
    }
}

export type CUnsafePair<S, T> = {
    fst: S,
    snd: T
}

export function CUnsafePair<S, T>(fst: S, snd: T): CUnsafePair<S, T> {
    // TODO: implement me
    //console.log(fst,snd)
    //function UnsafePair(fst2:S,snd2: T): 
    return {fst: fst, snd: snd};
    
    
}

In [ ]:

class BetterPair<S, T> {
    private fst: S;
    private snd: T;

    constructor(fst: S, snd: T) {
        this.fst = fst;
        this.snd = snd;
    }

    getFst(): S {
        return this.fst;
    }

    getSnd(): T {
        return this.snd;
    }
}

type CBetterPair<S, T> = {
    getFst: () => S,
    getSnd: () => T
}
function CBetterPair<S, T>(fst: S, snd: T): CBetterPair<S, T> {
    // TODO: implement me
    //throw Error("TODO");
    const bPair = new BetterPair(fst,snd);
    //return  BetterPair(bPair.getFst,bPair.getSnd)
    let _fst = fst
    let _snd = snd
    function _getFst(){
        return fst;
    }
    function _getSnd(){
        return snd;
    }
    return {
        getFst: _getFst, 
        getSnd: _getSnd
    }
        
        
}


In [ ]:

class Pair<S, T> {
    private fst: S;
    private snd: T;

    constructor(fst: S, snd: T) {
        this.fst = fst;
        this.snd = snd;
    }

    getFst(): S {
        return this.fst;
    }

    getSnd(): T {
        return this.snd;
    }

    setSnd(snd: T): void {
        this.snd = snd;
    }
}

type CPair<S, T> = {
    getFst: () => S,
    getSnd: () => T,
    setSnd: (snd: T) => void
};

function CPair<S, T>(fst: S, snd: T): CPair<S, T> {
    // TODO: implement me
    //throw Error("TODO");
    //const pair = new Pair(fst,snd);
    //return new Pair(fst,snd);
    function _getFst(){
        return fst;
    }
    function _getSnd(){
        return snd;
    }
    function _setSnd(snd:T){
        return snd
    }
    return {
        getFst: _getFst, 
        getSnd: _getSnd,
        setSnd: _getSnd
    }
}


In [ ]:
function arrayOfArithmeticFunctions(names: ("plus" | "minus" | "times" | "divide")[]): ((x: number, y: number) => number)[] {
    // TODO: implement me
    let numbers: ("plus" | "minus" | "times" | "divide")[] = []
    for (var i = 0; i < names.length; i++) {
        numbers.push(names[i])
        //console.log(numbers[i])
    }
    if(numbers[0]=="plus"){
        //console.log("yest0")
        //return numbers[0](((x: number, y: number) => x+y))
    }
    else if(numbers[1]=="minus"){
        //console.log("yest1")
    }
    else if(numbers[2]=="times"){
        //console.log("yest2")
    }
    else if(numbers[3]=="divide"){
        //console.log("yest3")
    }
    
    return [];
    
}
const fnArr = arrayOfArithmeticFunctions(["plus", "minus", "times"])
console.log(fnArr[0])
